In [ ]:
import os
import matplotlib.table
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from utils_experimentation import SolverFormatInfo, Convergence_Experiment

from bokeh.palettes import Category20
cat20 = Category20[20]

In [ ]:
processed_data_population_pkl_path = "C:\\Users\\dosre\\dev\\thesis-data\\experimentation\\output_data\\processed_data_population.pkl"
output_plot_root_dir_path = "C:\\Users\\dosre\\dev\\thesis-data\\experimentation\\support_plots"
os.makedirs(output_plot_root_dir_path, exist_ok=True)

In [ ]:
processed_data_population = pd.read_pickle(processed_data_population_pkl_path)

In [ ]:
experiments_to_plot = [
    ("FEM_3D_thermal1", "unpreconditioned", 30),
    ("marine1", "unpreconditioned", 30)
]

In [ ]:
for matrix, setup, inner_iter in experiments_to_plot:
    
    convergence_data_to_plot = processed_data_population.loc[
        (processed_data_population["matrix"] == matrix) &
        (processed_data_population["setup"] == setup) &
        (processed_data_population["inner_iter"] == inner_iter)
    ]

    plot_file_path = os.path.join(
        output_plot_root_dir_path,
        f"{matrix}_{setup}_inner_iter_{inner_iter:03d}.jpg"
    )

    conv_experiments = []
    for row in convergence_data_to_plot.iterrows():
        row = row[1]
        conv_experiments.append(Convergence_Experiment(row.solver, row.file_paths))

    fig, ax = plt.subplots(figsize=(6.4, 3.6))
    
    for exp_data in conv_experiments:
        if exp_data.id in SolverFormatInfo.fp_ids:
            exp_data.plot_convergence_data(ax)

    ax.legend()
    ax.semilogy()
    ax.grid()
    ax.set_ylabel("$\\frac{|| b-Ax_{i}||_{2}}{||b-Ax_{0}||_{2}}$")
    ax.set_xlabel("Inner Iteration")
    
    fig.suptitle(f"{matrix} {setup} GMRES({inner_iter}) Convergence")
    
    plt.savefig(plot_file_path, bbox_inches="tight")

    plt.show()

    plt.close()
    